In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

In [ ]:
# Define paths to your dataset
train_data_dir = '/content/drive/MyDrive/Glaucoma annoted/training'
test_data_dir = '/content/drive/MyDrive/Glaucoma annoted/validation'

In [ ]:
# Define batch size and number of epochs
batch_size = 16
epochs = 150
input_shape = (224, 224)  # Assuming images are resized to 224x224


print(f"Batch size: {batch_size}")

Batch size: 16


In [ ]:
# Data Augmentation and Image Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='binary'  # Since we use binary_crossentropy loss
)

Found 735 images belonging to 2 classes.


In [ ]:
# Flow validation images in batches using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='binary'
)

Found 116 images belonging to 2 classes.


In [ ]:
# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
print(f"Number of samples in training data: {train_generator.samples}")
print(f"Batch size: {batch_size}")

Number of samples in training data: 735
Batch size: 16


In [ ]:

# Calculate the total data count for training and validation datasets
total_train_samples = train_generator.samples
total_validation_samples = validation_generator.samples

print(f"Total training samples: {total_train_samples}")
print(f"Total validation samples: {total_validation_samples}")

Total training samples: 735
Total validation samples: 116


In [ ]:
# Add custom layers on top of ResNet50
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=10,
    epochs=epochs,
    validation_data=validation_generator
)


Epoch 1/150
10/10 [==============================] - 82s 8s/step - loss: 1.0111 - accuracy: 0.5625 - val_loss: 0.6937 - val_accuracy: 0.5690
Epoch 2/150
10/10 [==============================] - 57s 6s/step - loss: 0.6988 - accuracy: 0.6062 - val_loss: 0.6489 - val_accuracy: 0.5690
Epoch 3/150
10/10 [==============================] - 59s 6s/step - loss: 0.6807 - accuracy: 0.5688 - val_loss: 0.6422 - val_accuracy: 0.8103
Epoch 4/150
10/10 [==============================] - 76s 8s/step - loss: 0.6700 - accuracy: 0.6062 - val_loss: 0.6358 - val_accuracy: 0.6638
Epoch 5/150
10/10 [==============================] - 57s 6s/step - loss: 0.6661 - accuracy: 0.5472 - val_loss: 0.5979 - val_accuracy: 0.8879
Epoch 6/150
10/10 [==============================] - 60s 6s/step - loss: 0.6447 - accuracy: 0.5849 - val_loss: 0.6278 - val_accuracy: 0.5948
Epoch 7/150
10/10 [==============================] - 59s 6s/step - loss: 0.6037 - accuracy: 0.7233 - val_loss: 0.6354 - val_accuracy: 0.5690
Epoch 8/150
1

KeyboardInterrupt: 

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation loss: {loss:.4f}, Validation accuracy: {accuracy:.4f}')

# Plot training history
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.tight_layout()
plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
